In [ ]:
"""
__author__ = "Tereza Novakova"
__credits__ = ["Tereza Novakova"]
__license__ = "GPL"
__version__ = "1.0.1"
__maintainer__ = "Tereza Novakova"
__email__ = "teznovakova@gmail.com"
__status__ = "Production"
"""

In [5]:
# -*- coding: utf-8 -*-

import os #inter
import tifffile as tf
import numpy as np
import rasterio
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg
import earthpy.spatial as es
import pycrs
import json
import pprint 
from scipy import ndimage

%run functions.ipynb
%run CLIP_DATA.ipynb

In [ ]:
#Folder with subfolders for images
in_folder = r'C:\Users\Tereza\Documents\UP\DP\SNIMKY\INPUT\SNIMKY_L8_BREZEN_RIJEN'

#Outputfolder for final images
out_folder = r'C:\Users\Tereza\Documents\UP\DP\SKRIPTY\TEST'

#folder for intermediate results
trash = r'C:\Users\Tereza\Documents\UP\DP\SKRIPTY\TEST\trash'

In [ ]:
#boudning box around the chosen area in degrees
bbox= box(17.177762, 49.56004, 17.321102, 49.622711)

#chosen epsg for rasters
epsg_code = 32633


In [3]:
#Find all paths with valid sufix (TIF for L8 and JP2 for S2/S3)
def find_path(input_folder, file_name):
    
    path_list_folder = []
    for root, dirs, files in os.walk(input_folder, topdown=False):
        #vytiskne vsechno v input_folder
        #pprint.pprint(root)
        
        #projde vsechny soubory
        for name in files:
            #projdi vsechny nazvy
            #if file_name in name:
            if name.endswith(file_name) :
                #pokud obsahuje nazev z file_name
                #if suffix in name:
                    #seznam cest ke vsem suborum
                path_list_folder.append(os.path.join(root, name))
        #pprint.pprint(path_list_folder)
    return path_list_folder

#List of strings paths to Landsat Images
landsat_tif = find_path(in_folder, ".TIF")
#pprint.pprint(landsat_tif)

L8_dict = {}
#List of Landsat paths with valid name
for landsat_path in landsat_tif:
    landsat_name = os.path.basename(landsat_path)
    
    #Split the valid files with '_', then make them into a key in a dictionary
    if 'LC08_L1TP' in landsat_name:
    
        #List of each part between '_' -> ['LC08', 'L1TP', '189026', '20200801', '20200807', '01', 'T1', 'B1.TIF']
        #Make keys for dictionary, consists of unique sensing dates
        date = landsat_name.split('_')[3]
        
        #Create dictionary with date (key) and path string to each band (value) -> filtering data by sensing date
        #print(landsat_path)
        if date in L8_dict:
            L8_dict[date].append(landsat_path)
        else:
            L8_dict[date] = [landsat_path]
#pprint.pprint(L8_dict)



In [4]:
def find_path(input_folder, file_name):
    path_list_folder = []
    for root, dirs, files in os.walk(input_folder, topdown=False):
        for name in files:
            if name.endswith(file_name) :
                path_list_folder.append(os.path.join(root, name))
    return path_list_folder


#List of strings paths to Landsat Texts
landsat_txt = find_path(in_folder, "MTL.txt")
#pprint.pprint(landsat_txt)



['C:\\Users\\Tereza\\Documents\\UP\\DP\\SNIMKY\\INPUT\\SNIMKY_L8_BREZEN_RIJEN\\01082020\\LC08_L1TP_189026_20200801_20200807_01_T1_MTL.txt',
 'C:\\Users\\Tereza\\Documents\\UP\\DP\\SNIMKY\\INPUT\\SNIMKY_L8_BREZEN_RIJEN\\02042020\\LC08_L1TP_190025_20200402_20200410_01_T1_MTL.txt',
 'C:\\Users\\Tereza\\Documents\\UP\\DP\\SNIMKY\\INPUT\\SNIMKY_L8_BREZEN_RIJEN\\02092020\\LC08_L1TP_189026_20200902_20200906_01_T1_MTL.txt',
 'C:\\Users\\Tereza\\Documents\\UP\\DP\\SNIMKY\\INPUT\\SNIMKY_L8_BREZEN_RIJEN\\04052020\\LC08_L1TP_190025_20200504_20200509_01_T1_MTL.txt',
 'C:\\Users\\Tereza\\Documents\\UP\\DP\\SNIMKY\\INPUT\\SNIMKY_L8_BREZEN_RIJEN\\08082020\\LC08_L1TP_190025_20200808_20200821_01_T1_MTL.txt',
 'C:\\Users\\Tereza\\Documents\\UP\\DP\\SNIMKY\\INPUT\\SNIMKY_L8_BREZEN_RIJEN\\09092020\\LC08_L1TP_190025_20200909_20200918_01_T1_MTL.txt',
 'C:\\Users\\Tereza\\Documents\\UP\\DP\\SNIMKY\\INPUT\\SNIMKY_L8_BREZEN_RIJEN\\11042020\\LC08_L1TP_189026_20200411_20200422_01_T1_MTL.txt',
 'C:\\Users\\Tereza\

In [ ]:
for dates, list_of_paths in L8_dict.items():
    list_of_paths_clipped = []
    
    txt_path = ''
    
    L8_metadata = {}
    
    # read metadata text file and store values for one day in dict
    for path in list_of_paths:
        if 'B1.TIF' in path:
            txt_path = path.replace('B1.TIF', 'MTL.txt')
            #print(txt_path)
            metadata_file = open(txt_path, 'r')
            for line in metadata_file:
                #print(line)
                # last value is just END without any value
                if '=' in line:
                    key, value = line.strip().split(' = ')
                    #print(key_value)
                    L8_metadata[key] = value                                 
            metadata_file.close()
            # after find B1.TIF exit the loop
            break
    
           
    for path in list_of_paths:
            #print(txt_path)
            
        image_name = os.path.basename(path).replace('.TIF', '')
        #print(list_of_paths)
        out_path = trash + '\\' + image_name + '_Clipped.TIF'
        #print(out_path)
        ClipSatelliteImage(path, out_path, bbox, epsg_code)
        list_of_paths_clipped.append(out_path)
    #print(list_of_paths_clipped)
    
    
    for path in list_of_paths_clipped:
        #print(path)  
        if 'B1_' in path:
            b1 = path
        if 'B2' in path:
            b2 = path
        if 'B3' in path:
            b3 = path
        if 'B4' in path:
            b4 = path
        if 'B5' in path:
            b5 = path
        if 'B6' in path:
            b6 = path
        if 'B7' in path:
            b7 = path
        if 'B10' in path:
            b10 = path
            radiance_MULT_B10 = float((L8_metadata['RADIANCE_MULT_BAND_10']))
            radiance_ADD_B10 = float((L8_metadata['RADIANCE_ADD_BAND_10']))
            

    toa_path_b10 = fTOA_Rad(b10, radiance_MULT_B10, radiance_ADD_B10, out_folder,'TOARad_b10_' + dates)  
    BriTemp_path = fBT(toa_path_b10, K1_CONSTANT_BAND_10, K2_CONSTANT_BAND_10, out_folder, 'BriTemp_' + dates)
    ndvi_path = fndvi(b4, b5, out_folder, 'NDVI_' + dates)
    VegCov_path = fVc(ndvi_path, out_folder, 'VegCov_' + dates)
    SurfEmis_path = fEmis(VegCov_path, ndvi_path, b4, out_folder, 'SurfEmis_' + dates)
    LST_path = fLST(BriTemp_path, SurfEmis_path, out_folder, 'LST_' + dates)
    